In [12]:
from tensorflow.keras import Model
from tensorflow.keras.layers import Input, MaxPooling1D, Conv1D, Dropout, Dense, Flatten
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.backend import clear_session
from tensorflow.keras.callbacks import EarlyStopping

import matplotlib.pyplot as plt
from transformer import TransformerBlock, TokenAndPositionEmbedding
from utils import load_dataset, train
from einops import rearrange

<IPython.core.display.Javascript object>

In [10]:
maxlen = 1000
sample_size = 4
embed_dim = 4  # Embedding size for each token
num_heads = 2  # Number of attention heads
ff_dim = 64  # Hidden layer size in feed forward network inside transformer

clear_session()

inputs = Input(shape=(1000, 4))

x = Conv1D(128, kernel_size=26, activation="relu")(inputs)
x = MaxPooling1D(pool_size=13, strides=13)(x)
x = Dropout(0.2)(x)
# x = TokenAndPositionEmbedding(maxlen, sample_size, embed_dim)(x)
x = TransformerBlock(128, num_heads, ff_dim)(x)
x = Dropout(0.2)(x)
x = Dense(925, activation="relu")(x)
outputs = Dense(1, activation="softmax")(x)

model = Model(inputs, outputs, name="conv_transformer")

model.summary()

Model: "conv_transformer"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_1 (InputLayer)         [(None, 1000, 4)]         0         
_________________________________________________________________
conv1d (Conv1D)              (None, 975, 128)          13440     
_________________________________________________________________
max_pooling1d (MaxPooling1D) (None, 75, 128)           0         
_________________________________________________________________
dropout (Dropout)            (None, 75, 128)           0         
_________________________________________________________________
transformer_block (Transform (None, 75, 128)           83136     
_________________________________________________________________
dropout_3 (Dropout)          (None, 75, 128)           0         
_________________________________________________________________
dense_6 (Dense)              (None, 75, 925)      

<IPython.core.display.Javascript object>

In [14]:
es = EarlyStopping(monitor="val_loss", patience=30)
optimizer = Adam(lr=1e-4)

epochs = 1000
validation_freq = 1
n = 5
file = f"MA0035_{n}000"
boundary = 5000

<IPython.core.display.Javascript object>

In [15]:
X_train, y_train, X_test, y_test = load_dataset(file=file, labels="binlabels")
X_train = rearrange(X_train, "w h c -> w c h")
X_test = rearrange(X_test, "w h c -> w c h")

# np.where(y_train > boundary, 0, 1)
# np.where(y_test > boundary, 0, 1)

<IPython.core.display.Javascript object>

In [16]:
model.compile(optimizer=optimizer, loss="binary_crossentropy", metrics=["acc"])

<IPython.core.display.Javascript object>

In [19]:
model = train(
    dataset=(X_train, y_train, X_test, y_test),
    model=model,
    epochs=epochs,
    validation_freq=validation_freq,
    callbacks=[es],
)

KeyboardInterrupt: 

<IPython.core.display.Javascript object>

In [ ]:
model.evaluate(X_test, y_test)

In [ ]:
model.save('../model_saves/transformer-model.h5')